In [ ]:
import numpy as np
import tifffile as tif
import os

# data manager and analysis
import vodex as vx
import numan as nu
import pandas as pd

Get the project directory:

In [ ]:
# please provedi FULL, not relative, path to the folder 
project_folder = "/home/ply/repos/numan_dev/numan/data/hz09"
project = nu.Project(project_folder)

We will also set the processed directory as our working directory, this step is important , since all the paths later are relative to this folder. Verify that the output of the cell is the "processed" folder inside your project folder. 

In [ ]:
project.activate("processed")
os.getcwd()

# Load experiment: 

* For now we still need the raw experiment since we will be loading data around from it to process. 
* we will also need the information about the truncated experiment that we saved to stimuli_truncated_timelines.csv in the previous notebook.

In [ ]:
experiment = vx.Experiment.load("experiment_raw.db")

# grab the volumes around the stimuli only:
stimuli_volumes_df = pd.read_csv("stimuli_truncated_timelines.csv", index_col = False)
stimuli_volumes_extended = stimuli_volumes_df["volumes"].to_numpy()

stimuli_volumes_extended[0:9]

From now on we will only use the volumes that we defined above. This will save us a lot of computation time.

# Denoising

You need to provide the offset file for denoising

In [ ]:
# this file is the same for all experiments with this camera settings
# you can get the file at https://github.com/LemonJust/numan/tree/main/notebooks/data/denoising_2023_hz
offset_file = "/home/ply/repos/numan_dev/numan/notebooks/data/denoising_2023_hz/calibration_offset.tif"

# 200 volumes takes about 12 Gb of RAM and takes about 10 minutes to process, 
# but 1 file with 200 volumes is about 4Gb , 
# so shouldn't increase it above 200
batch_size = 200 # in volumes

In [ ]:
# takes about 3 minutes to process on beefsy
project.create("processed/removed_offset")

nu.Preprocess(experiment).remove_offset('removed_offset',batch_size, volumes = stimuli_volumes_extended, offset_file = offset_file,
                                        verbose=True )

Update the experiment to use the denoised files:

In [ ]:
# update files info to use the new removed_offset folder instead of the original raw data
data_dir = "removed_offset"

# volumes info is the same as in the original experiment
frames_per_volume = experiment.frames_per_volume
starting_slice = experiment.starting_slice

# close the old experiment
experiment.close()
# create new experiment
experiment = vx.Experiment.from_dir(data_dir, frames_per_volume, starting_slice, verbose=True)
# add time annotation 
experiment.add_annotations_from_volume_annotation_df(stimuli_volumes_df)

# for now there will be no description of the labels when adding them from the volume annotation df
# so you will see description : None for all labels in the labels_df
# will add this option in later versions of vodex
experiment.labels_df

If everything looks good, save the experiment to the processed folder.

In [ ]:
experiment.save("experiment_truncated_denoised.db")

# Drift correction
* will only register the volumes that we defined above for the truncated experiment.
* uses denoised data
* for now registers to the 1000th volume in the movie, might add a smarter way to pick the template later
* using Affne registration
* takes about 2.5 hours on beefsy 

In [ ]:
experiment = vx.Experiment.load("experiment_truncated_denoised.db")

In [ ]:
save_dir = "drift_corrected_af_to1000"
project.create(f"processed/{save_dir}")

# the RAM doesn't depend much on the batch size (needs baout 4 - 5 Gb) as each volume pair is loaded separately for the registration
# takes about 14 min for 200 volumes, so 2.5 hours for the whole experiment (2160 volumes)
batch_size = 200 # in volumes
template = 1000 # approx in the middle of the truncated experiment
spacing_xyz = [1.62, 1.62, 3.83] # in um

nu.Preprocess(experiment).drift_correct_naive(save_dir,batch_size, spacing_xyz, registration_type = "Affine", template = template, verbose=True)

Update the experiment to use the drift corrected files:

In [ ]:
# update files info to use the new removed_offset folder instead of the original raw data
data_dir = "drift_corrected_af_to1000"

# volumes info is the same as in the original experiment
frames_per_volume = experiment.frames_per_volume
starting_slice = experiment.starting_slice

# close the old experiment
experiment.close()
# create new experiment
experiment = vx.Experiment.from_dir(data_dir, frames_per_volume, starting_slice, verbose=True)
# add time annotation 
experiment.add_annotations_from_volume_annotation_df(stimuli_volumes_df)

# for now there will be no description of the labels when adding them from the volume annotation df
# so you will see description : None for all labels in the labels_df
# will add this option in later versions of vodex
experiment.labels_df

If everything looks good, save the experiment to the processed folder.

In [ ]:
experiment.save("experiment_truncated_drift_corrected.db")

Save three slices for easy visualization:

In [ ]:
# create directory for slices
project.create("processed/drift_corrected_slices")

# get the slices
# you can pick any slices you want, 
# they will be saved in the same file
# the order doesn't matter (vodex will load them smallest to largest)
# three slices are about 4 Gb for the whole movie
slices_id = [10, 17, 22]
slices = experiment.load_slices(slices_id)

# create file name according to the slices id
file_name = "drift_corrected_slices/dc_slices"
for slice_id in slices_id:
    file_name = file_name + "_" + str(slice_id)

# save the slices
tif.imwrite(file_name + ".tif",
            slices.astype(np.int16), 
            shape=slices.shape, metadata={'axes': 'TZYX'}, imagej=True)

experiment.close()